# Estimation of Pi Using Random Numbers

Adapted from HPC Carpentries lesson [HPC Parallelisation For Novices](https://www.hpc-carpentry.org/hpc-parallel-novice/).

We will use the Julia programming language to estimate the value of $\pi$.

## The Problem

Consider a square enclosing a quarter of a circle with radius $R$. The area of the square is $A_s$ is <br>
    
$A_s = R^2$

and the area of the quarter circle $A_{qc}$ is
    
$A_{qc} = \frac{1}{4} \pi R^2$.

Substituting for $R^2$, one arrives at

$A_{qc} = \frac{1}{4} \pi A_s$

$\pi = 4 \frac{A_{qc}}{A_s}$

meaning that $\pi$ is equal to four times the ratio of the areas of the quarter circle to the square. Therefore, we can use random numbers to calculate the value of $\pi$ by constructing a unit square, throwing random points into the square, and counting the number of points which land in the quarter circle.

![image](estimate_pi.svg "Title")

The number of points which land in the quarter circle divided by the total number of random points we threw should give us an approximation of the ratio of the areas (with the accuracy increasing with the number of total points used).

We now solve this problem using various methodologies to illustrate the power of vectorization and parallelization.

## Using For Loops

The simplest method involves using a single for loop to generate the random points and determine whether the point falls within the circle.

Let us now put the above code into a function so that we can call it later to do comparisons against other methods.

In [6]:
function serial_for_loop_pi(numberOfPoints)
    numberOfCirclePoints = 0
    x = rand(Float64,numberOfPoints)
    y = rand(Float64,numberOfPoints)
    for i in 1:numberOfPoints
        #x = rand()
        #y = rand()
        r = sqrt(x[i]*x[i]+y[i]*y[i])
        if r <= 1
            numberOfCirclePoints += 1
        end
    end
    return(4*numberOfCirclePoints/numberOfPoints)
end

serial_for_loop_pi (generic function with 1 method)

## Using Built-In Vectors

In [7]:
## Using Vectors
function serial_vector_pi(numberOfPoints)
    numberOfCirclePoints = 0
    x = rand(Float64,numberOfPoints)
    y = rand(Float64,numberOfPoints)
    r = map(sqrt,x.^2+y.^2)
    numberOfCirclePoints = count(r->(r<=1),r)
    return(4 * numberOfCirclePoints / numberOfPoints)
end

serial_vector_pi (generic function with 1 method)

## Using Multithreading

In [12]:
function thread_pi(numberOfPoints)
    numberOfCirclePoints = 0
    #x = rand(Float64,numberOfPoints)
    #y = rand(Float64,numberOfPoints)
    Threads.@threads for i in 1:numberOfPoints
        x = rand(Float64)
        y = rand(Float64)
        r = sqrt(x^2+y^2)
        if r <= 1.0
            numberOfCirclePoints += 1
        end
    end
    pi = 4*numberOfCirclePoints/numberOfPoints
    print("Pi = $pi")
    return(pi)
end

function thread_for_loop_pi(numberOfPoints)
    numberOfCirclePoints = 0
    x = rand(Float64,numberOfPoints)
    y = rand(Float64,numberOfPoints)
    r = rand(Float64,numberOfPoints)
    Threads.@threads for i in 1:numberOfPoints
        #x = rand()
        #y = rand()
        r[i] = sqrt(x[i]*x[i]+y[i]*y[i])
        if r[i] <= 1
            numberOfCirclePoints += 1
        end
    end
    return(4*numberOfCirclePoints/numberOfPoints)
end

thread_for_loop_pi (generic function with 1 method)

In [19]:
thread_for_loop_pi(99999)

1.011290112901129

In [15]:
serial_for_loop_pi(99999)

3.143671436714367

In [10]:
serial_vector_pi(99999)

3.138711387113871